In [69]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#from joblib import dump, loada
import joblib
import csv

In [70]:
# Carga datos y crea variables dummy
df = pd.read_csv('juadores_fantasy.csv.csv')
posicion__orig=df['posicion']
# Guarda nombre original de columna
pos_col = 'posicion'

#categorical_cols = ['posicion', 'equipo']  
#df = pd.get_dummies(df, columns=categorical_cols)

In [71]:
# Codifica posicion y equipo a variables dummy
df = pd.get_dummies(df, columns=['posicion', 'equipo'])

In [72]:
# Obtén los nombres de las columnas dummy creadas
pos_dummies = [c for c in df.columns if c.startswith('posicion_')]

In [73]:
#Solicita entrada de posición
posicion = "delantero"


In [74]:
# Filtra usando una de las columnas dummy
df_filtrado = df[df[pos_dummies[0]] == 1]

In [75]:
# Separa las features de las variables a predecir  
X = df.drop(['Nombre', 'Total_puntos', 'Precio'], axis=1)
y_puntos = df['Total_puntos'] 
y_precio = df['Precio']

In [76]:
# Divide los datos en entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.3, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.3, random_state=42)

In [77]:
# Entrena un modelo para predecir puntos
rf_puntos = RandomForestRegressor()
rf_puntos.fit(X_train, y_puntos_train)

RandomForestRegressor()

In [78]:
# Entrena un modelo para predecir precio 
rf_precio = RandomForestRegressor(n_estimators=100)
rf_precio.fit(X_train, y_precio_train)

RandomForestRegressor()

In [79]:
# Realiza predicciones en los datos de test
y_puntos_pred = rf_puntos.predict(X_test)
y_precio_pred = rf_precio.predict(X_test)

In [80]:
# Evalúa los modelos
print("Puntos R2 Score:", rf_puntos.score(X_test, y_puntos_test))
mse = mean_squared_error(y_puntos_test, y_puntos_pred) 
print("Puntos MSE:", mse)

print("Precio R2 Score:", rf_precio.score(X_test, y_precio_test))
mse = mean_squared_error(y_precio_test, y_precio_pred)
print("Precio MSE:", mse)

Puntos R2 Score: 0.7168317209974702
Puntos MSE: 348.7827714681113
Precio R2 Score: 0.679017830672532
Precio MSE: 11643722982897.62


In [81]:
# Obtén las predicciones ordenadas para encontrar los mayores
indices_puntos = y_puntos_pred.argsort()[::-1]
indices_precio = y_precio_pred.argsort()[::-1]

In [82]:
print(f"\n5 jugadores {posicion} con mayor puntaje predicho:")

for i in indices_puntos[:10]:

  nombre = df.loc[i,'Nombre']
  posicion= posicion__orig.iloc[i]
  puntos = y_puntos_pred[i]
  precio = y_precio_pred[i]

  print(nombre, puntos, posicion, precio)


5 jugadores delantero con mayor puntaje predicho:
R. Alcaraz 137.19 mediocentro 19451160.0
R. Lewandowski 136.73 delantero 17647940.0
Á. Morata 123.48 delantero 16779560.0
S. Arribas 122.43 mediocentro 15877950.0
L. Rioja 116.55 mediocentro 15275750.0
M. Llorente 115.26 mediocentro 16998870.0
V. Muriqi 115.25 delantero 16736630.0
F. Jörgensen 112.11 portero 13745800.0
D. García 110.73 defensa 16106320.0
E. Loiodice 97.78666666666668 mediocentro 14594570.0


In [83]:
with open('sugerencia.csv', 'w', newline='') as file:
  writer = csv.writer(file)

  writer.writerow(['Nombre', 'Puntos', 'Posicion','Precio']) 
  
  for i in indices_puntos[:10]:
    nombre = df.loc[i,'Nombre']
    posicion= posicion__orig.iloc[i]
    puntos = y_puntos_pred[i] 
    precio = y_precio_pred[i]
    
    writer.writerow([nombre, puntos, posicion, precio])

In [84]:
#joblib.dump(rf_puntos, 'random_forest.pkl')

In [85]:
#print(f"\n2 jugadores {posicion} con mayor precio predicho:")
#for i in indices_precio[:10]:
#  print(df.loc[i,'Nombre'], y_precio_pred[i])